In [9]:
#utils
import emoji, re, string, time, os
from utils import getTestMetrics, getTopUsers
from utils import preprocess
import pandas as pd
import numpy as np
from scipy.stats import randint
import pickle
import math

#nlp
import nltk
from nltk.corpus import stopwords
import spacy
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

#dataviz
import matplotlib.pyplot as plt
import seaborn as sns

#features
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

#models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble  import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn import tree

#data balancing
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# K-Fold
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

%matplotlib inline
sns.set(style="darkgrid")

# Colecting high quality, non-ambiguos data

**Result**: wpp_to_label_2018_ambiguity

In [5]:
# df_first_label = pd.read_csv('labels/2018/wpp_first_labels.csv')
# df_sec_label = pd.read_csv('labels/2018/wpp_second_labels.csv')
# # select only high quality data
# def get_revision(x):
#     if x == '1':
#         return 1
#     else:
#         return 0
    
# df_first_label = df_first_label[['MSG', 'COUNT', 'DISINFORMATION', 'FACT_CHECKING']]
# df_first_label['revision'] = [get_revision(x) for x in df_first_label['FACT_CHECKING']]
# df_first_label

# df_sec_label = df_sec_label[['MENSAGEM','DESINFORMACAO','REVISAR']]
# df_sec_label.dropna(subset=['REVISAR'])

### Assign revision tag from first to second

In [6]:
# vectorizer = TfidfVectorizer()
# X_first = vectorizer.fit_transform(df_first_label['MSG'])
# revision = df_sec_label['REVISAR']
# X_sec = vectorizer.transform(df_sec_label['MENSAGEM'])
# first_revision = list(df_first_label['revision'])

# for i,vec in enumerate(X_sec):
    
#     if math.isnan(revision[i]): 
#         cosine_similarities = linear_kernel(vec, X_first).flatten()
#         ix = np.argmax(cosine_similarities)            
#         cos_sim = (np.max(cosine_similarities))
        
#         if cos_sim > 0.85:
#             revision[i] = first_revision[ix]
# print (revision.isna().sum())
# df_sec_label['ambiguous'] = revision
# df_sec_label = df_sec_label.sort_values(by='ambiguous')
# df_sec_label.to_csv('labels/2018/wpp_to_label_2018_ambiguity.csv',index=False)


# Assign labels
**result:** data/2018/high quality labels/fakeWhatsApp.BR_2018.csv

In [7]:
# df_labels = pd.read_csv('labels/2018/wpp_to_label_2018_ambiguity.csv')
# df_labels.columns = ['text','misinformation','revision','ambiguous']
# df = pd.read_csv('data/2018/fakeWhatsApp.BR_2018.csv')

# ## clear column (-1 == empty)
# misinformation = [-1]*len(df)
# labels_mis = df_labels['misinformation']
# revision = [-1]*len(df)
# labels_rev = df_labels['ambiguous']


# vectorizer = TfidfVectorizer()
# X_df = vectorizer.fit_transform(df['text'])

# X_labels = vectorizer.transform(df_labels['text'])
# ambiguous = list(df_labels['ambiguous'])

# for i,x in enumerate(X_labels): 
#     cosine_similarities = linear_kernel(x, X_df).flatten()
#     ix = np.argmax(cosine_similarities)            
#     max_sim = (np.max(cosine_similarities))        
#     if max_sim > 0.85:
#         misinformation[ix] = labels_mis[i]
#         revision[ix] = labels_rev[i]
        
# df['misinformation'] = misinformation
# df['ambiguity'] = revision


# df['preprocessed_text'] = [preprocess(t) for t in df['text']]
# df['preprocessed_text_stemmed'] = [preprocess(t, stem=True) for t in df['text']]
# df['preprocessed_text_lemma'] = [preprocess(t, lemma=True) for t in df['text']]
# df_pre_texts = df[['preprocessed_text','preprocessed_text_stemmed','preprocessed_text_lemma']]
# df_pre_texts.to_csv('data/2018/high quality labels/fakeWhatsApp.BR_2018_preprocessed_texts.csv')

# feat= ['id', 'date', 'hour', 'ddi', 'country', 'country_iso3', 'ddd', 'state',
#        'group', 'midia', 'url', 'characters', 'words', 'viral', 'sharings',
#        'text', 'misinformation', 'ambiguity']
# df = df[feat]
# df.to_csv('data/2018/high quality labels/fakeWhatsApp.BR_2018.csv',index=False)


C:\Users\lucas.carneiro\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
len(df)

282601

In [12]:
df_users = getTopUsers(df, None)
df_users.to_csv('data/2018/user_features.csv',index = False)

NameError: name 'df' is not defined

In [7]:
# sample = df[df['misinformation']==1]['text'][0:100]
# for s in sample:
#     print('*******')
#     print(s)
#     print('------')
#     print(preprocess(s,stem=True))

# Load and select data

In [2]:
df = pd.read_csv('data/2018/high quality labels/fakeWhatsApp.BR_2018.csv')
df.tail()

C:\Users\lucas.carneiro\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,date,hour,ddi,country,country_iso3,ddd,state,group,midia,url,characters,words,viral,shares,text,misinformation,preprocessed_text,ambiguity
146918,4806790986300909984,28/10/18,12:29,?,?,?,?,?,2018_59,0,0,60,11,0,1,"O pior é compartilhar isso\nPara cara, isso va...",-1,"pior compartilhar , vai foder conosco",-1.0
146919,4806790986300909984,28/10/18,12:29,?,?,?,?,?,2018_59,0,0,29,6,0,1,Japão deu uma surra no Haddad,-1,japao surra haddad,-1.0
146920,-695233245073530941,28/10/18,14:26,?,?,?,?,?,2018_59,0,0,1,1,0,1,5,-1,[num],-1.0
146921,807720138466692852,28/10/18,16:48,?,?,?,?,?,2018_59,0,0,68,11,0,1,Como faz pra baixar esses desenhos\nTentei aqu...,-1,faz baixar desenhos tentei consegui . . . . 😂 ...,-1.0
146922,4806790986300909984,28/10/18,17:46,?,?,?,?,?,2018_59,0,0,19,5,0,1,Só começa dps das 7,-1,comeca dps [num],-1.0


In [ ]:
# data selection

# labelled
df = df[df['misinformation'] != -1]

# non-ambiguous
df = df[df['ambiguity'] == 0]

len(df)